In [4]:
import os
import json

# Entidades válidas
valid_labels = {
    'B-LOCAL', 'I-LOCAL',
    'B-ORGANIZACAO', 'I-ORGANIZACAO',
    'B-PESSOA', 'I-PESSOA',
    'B-TEMPO', 'I-TEMPO',
}


# Pasta base onde estão as partições
base_dir = "../Base de Dados/Paramopama/divisions/"
output_dir = "../Base de Dados/Paramopama/divisions/"
os.makedirs(output_dir, exist_ok=True)

# Função para converter BIO para JSON estruturado
def convert_bio_file_to_json(txt_path):
    with open(txt_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    samples = []
    tokens, labels = [], []

    for line in lines:
        line = line.strip()
        if not line:
            if tokens:
                samples.append((tokens, labels))
                tokens, labels = [], []
            continue
        parts = line.split()
        if len(parts) == 2:
            token, label = parts
            tokens.append(token)
            labels.append(label)
    
    # última frase
    if tokens:
        samples.append((tokens, labels))

    data_json = []
    for tokens, labels in samples:
        text = " ".join(tokens)
        entities = []
        i = 0
        while i < len(labels):
            if labels[i].startswith("B-"):
                label_type = labels[i][2:]
                entity_tokens = [tokens[i]]
                i += 1
                while i < len(labels) and labels[i].startswith("I-") and labels[i][2:] == label_type:
                    entity_tokens.append(tokens[i])
                    i += 1
                entities.append({
                    "text": " ".join(entity_tokens),
                    "label": label_type
                })
            else:
                i += 1
        data_json.append({
            "text": text,
            "entities": entities
        })
    
    return data_json

# Iterar sobre os 10 arquivos de divisão e salvar em JSON
for i in range(10):
    txt_file = os.path.join(base_dir, f"division_{i}.txt")
    json_file = os.path.join(output_dir, f"division_{i}.json")
    json_data = convert_bio_file_to_json(txt_file)
    with open(json_file, "w", encoding="utf-8") as f:
        json.dump(json_data, f, indent=2, ensure_ascii=False)
    print(f"Convertido: {txt_file} → {json_file}")



Convertido: ../Base de Dados/Paramopama/divisions/division_0.txt → ../Base de Dados/Paramopama/divisions/division_0.json
Convertido: ../Base de Dados/Paramopama/divisions/division_1.txt → ../Base de Dados/Paramopama/divisions/division_1.json
Convertido: ../Base de Dados/Paramopama/divisions/division_2.txt → ../Base de Dados/Paramopama/divisions/division_2.json
Convertido: ../Base de Dados/Paramopama/divisions/division_3.txt → ../Base de Dados/Paramopama/divisions/division_3.json
Convertido: ../Base de Dados/Paramopama/divisions/division_4.txt → ../Base de Dados/Paramopama/divisions/division_4.json
Convertido: ../Base de Dados/Paramopama/divisions/division_5.txt → ../Base de Dados/Paramopama/divisions/division_5.json
Convertido: ../Base de Dados/Paramopama/divisions/division_6.txt → ../Base de Dados/Paramopama/divisions/division_6.json
Convertido: ../Base de Dados/Paramopama/divisions/division_7.txt → ../Base de Dados/Paramopama/divisions/division_7.json
Convertido: ../Base de Dados/Par

In [3]:
import os
import json
import time
import glob
import requests
from tqdm import tqdm
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer
import torch

# Verifica se a GPU está disponível
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")
if torch.cuda.is_available():
    print(f"GPU disponível: {torch.cuda.get_device_name(torch.cuda.current_device())}")
else:
    print("Nenhuma GPU disponível, utilizando CPU.")

OLLAMA_MODEL_NAME = "qwen3:14b"
OLLAMA_URL = "http://localhost:11434/api/generate"

base_dir = "../Base de Dados/HAREM/divisions/first/json_divisions/"
division_files = [f"division_{i}.json" for i in range(10)]

def load_data(path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def get_all_labels():
    return ["LOCAL", "ORGANIZACAO", "PESSOA", "TEMPO", "ABSTRACAO", "ACONTECIMENTO", "COISA", "OBRA", "OUTRO", "VALOR"]

def select_minimal_few_shot(dataset):
    all_labels = set(get_all_labels())
    selected = []
    used_ids = set()
    label_to_items = {label: [] for label in all_labels}
    for item in dataset:
        for ent in item.get('entities', []):
            if ent['label'] in all_labels:
                label_to_items[ent['label']].append(item)
    for label in all_labels:
        for item in label_to_items[label]:
            if id(item) not in used_ids:
                selected.append(item)
                used_ids.add(id(item))
                break
    return selected

def truncate_text(text, max_tokens=150):
    return text[:max_tokens] + "..." if len(text) > max_tokens else text

def build_prompt(few_shot_examples, test_text, max_context_tokens=2048, buffer_tokens=256):
    system_prompt = """\
Tarefa: Extraia entidades nomeadas do texto abaixo e retorne apenas uma lista JSON válida.
Cada entidade deve ter os campos: "text" e "label".

- LOCAL
- ORGANIZACAO
- PESSOA
- TEMPO
- ABSTRACAO
- ACONTECIMENTO
- COISA
- OBRA
- OUTRO
- VALOR

Formato esperado:
[
  { "text": "em 1947", "label": "TEMPO"},
  { "text": "Congresso Nacional Indiano", "label": "ORGANIZACAO"}
]

"""
    prompt = system_prompt
    current_tokens = len(prompt.split())  # simples estimativa

    for ex in few_shot_examples:
        example_text = f"Texto: {truncate_text(ex['text'])}\nEntidades: {json.dumps(ex['entities'], ensure_ascii=False)}\n\n"
        example_tokens = len(example_text.split())
        if current_tokens + example_tokens + buffer_tokens > max_context_tokens:
            break
        prompt += example_text
        current_tokens += example_tokens

    test_section = f"Texto: {truncate_text(test_text)}\nEntidades:"
    prompt += test_section

    return prompt

def query_ollama(prompt, max_tokens=512):
    response = requests.post(
        OLLAMA_URL,
        json={
            "model": OLLAMA_MODEL_NAME,
            "prompt": prompt,
            "stream": False,
            "options": {
                "temperature": 0.3,
                "top_k": 40,
                "top_p": 0.9,
                "num_predict": max_tokens
            }
        }
    )
    response.raise_for_status()
    return response.json()["response"].strip()

# Limpa o relatório final
open("relatorio_folds.txt", "w").close()

labels = get_all_labels()
mlb = MultiLabelBinarizer(classes=labels)

folds_concluidos = {
    int(f.split("_")[-1].split(".")[0]) for f in glob.glob("resultados_fold_*.json")
}

all_reports = []

for i in range(10):
    if i in folds_concluidos:
        print(f"Fold {i} já processado. Pulando...")
        continue

    print(f"\nFold {i}")
    test_file = division_files[i]
    train_files = [f for j, f in enumerate(division_files) if j != i]

    test_data = load_data(os.path.join(base_dir, test_file))
    train_data = []
    for tf in train_files:
        train_data.extend(load_data(os.path.join(base_dir, tf)))

    few_shot = select_minimal_few_shot(train_data)
    print(f"Exemplos few-shot usados: {len(few_shot)}")

    print("Exemplos few-shot usados para este fold:")
    for ex in few_shot:
        print(f"Texto: {ex['text']}")
        print(f"Entidades: {json.dumps(ex['entities'], ensure_ascii=False)}")
        print("-" * 50)

    print(f"Total de exemplos no fold de teste: {len(test_data)}")
    print("-" * 50)

    with open(f"erros_fold_{i}.txt", "w", encoding="utf-8") as f_err:
        f_err.write(f"Erros do Fold {i}\n\n")

    y_true_list, y_pred_list = [], []

    for item in tqdm(test_data, desc=f"Processando Fold {i}"):
        start_time = time.time()
        prompt = build_prompt(few_shot, item['text'])

        try:
            output_text = query_ollama(prompt)
        except Exception as e:
            print(f"Erro na inferência: {e}")
            continue

        exec_time = time.time() - start_time
        print(f"Tempo de execução para exemplo: {exec_time:.4f} segundos")

        try:
            if not output_text.startswith("["):
                output_text = output_text[output_text.find("["):]
            pred_entities = json.loads(output_text)
            if not isinstance(pred_entities, list):
                pred_entities = []
        except Exception:
            pred_entities = []

        y_true_labels = [ent['label'] for ent in item.get('entities', []) if ent['label'] in labels]
        y_pred_labels = [ent['label'] for ent in pred_entities if isinstance(ent, dict) and 'label' in ent and ent['label'] in labels]

        y_true_list.append(y_true_labels)
        y_pred_list.append(y_pred_labels)

        if set(y_true_labels) != set(y_pred_labels):
            with open(f"erros_fold_{i}.txt", "a", encoding="utf-8") as f_err:
                f_err.write("Texto:\n")
                f_err.write(item['text'] + "\n")
                f_err.write("Entidades verdadeiras:\n")
                f_err.write(json.dumps(item.get('entities', []), ensure_ascii=False, indent=2) + "\n")
                f_err.write("Entidades previstas:\n")
                f_err.write(json.dumps(pred_entities, ensure_ascii=False, indent=2) + "\n")
                f_err.write("-" * 80 + "\n")

    y_true_bin = mlb.fit_transform(y_true_list)
    y_pred_bin = mlb.transform(y_pred_list)

    report = classification_report(y_true_bin, y_pred_bin, target_names=labels, output_dict=True, zero_division=0)
    all_reports.append(report)

    report_lines = [
        f"\n====== FOLD {i} ======",
        "Relatório por label:"
    ]

    for label in labels:
        report_lines.append(
            f"- {label}: precision={report[label]['precision']:.3f}, "
            f"recall={report[label]['recall']:.3f}, f1-score={report[label]['f1-score']:.3f}"
        )

    report_lines.extend([
        f"\nMacro Avg: precision={report['macro avg']['precision']:.3f}, recall={report['macro avg']['recall']:.3f}, f1-score={report['macro avg']['f1-score']:.3f}",
        f"Micro Avg: precision={report['micro avg']['precision']:.3f}, recall={report['micro avg']['recall']:.3f}, f1-score={report['micro avg']['f1-score']:.3f}",
        "=" * 50
    ])

    with open("relatorio_folds.txt", "a", encoding="utf-8") as f:
        f.write("\n".join(report_lines) + "\n")

    with open(f"resultados_fold_{i}.json", "w", encoding="utf-8") as f_json:
        json.dump({
            "fold": i,
            "report": report
        }, f_json, ensure_ascii=False, indent=2)

# ===== Relatório Final =====
final_report = {}
label_averages = {label: {"precision": 0, "recall": 0, "f1": 0} for label in labels}

loaded_reports = []
for i in range(10):
    try:
        with open(f"resultados_fold_{i}.json", "r", encoding="utf-8") as f_json:
            data = json.load(f_json)
            loaded_reports.append(data["report"])
    except FileNotFoundError:
        print(f"Aviso: resultado do fold {i} não encontrado. Pulando esse fold.")

for label in labels:
    precision = sum(r[label]['precision'] for r in loaded_reports if label in r) / len(loaded_reports)
    recall = sum(r[label]['recall'] for r in loaded_reports if label in r) / len(loaded_reports)
    f1 = sum(r[label]['f1-score'] for r in loaded_reports if label in r) / len(loaded_reports)
    label_averages[label] = {"precision": precision, "recall": recall, "f1": f1}

macro_precision = sum(r["macro avg"]["precision"] for r in loaded_reports) / len(loaded_reports)
macro_recall = sum(r["macro avg"]["recall"] for r in loaded_reports) / len(loaded_reports)
macro_f1 = sum(r["macro avg"]["f1-score"] for r in loaded_reports) / len(loaded_reports)

micro_precision = sum(r["micro avg"]["precision"] for r in loaded_reports) / len(loaded_reports)
micro_recall = sum(r["micro avg"]["recall"] for r in loaded_reports) / len(loaded_reports)
micro_f1 = sum(r["micro avg"]["f1-score"] for r in loaded_reports) / len(loaded_reports)

with open("relatorio_final.txt", "w", encoding="utf-8") as f:
    f.write("=== MÉDIAS DOS 10 FOLDS ===\n\n")
    for label, scores in label_averages.items():
        f.write(f"{label}: Precision={scores['precision']:.3f}, Recall={scores['recall']:.3f}, F1={scores['f1']:.3f}\n")
    f.write("\nMacro Average:\n")
    f.write(f"Precision={macro_precision:.3f}, Recall={macro_recall:.3f}, F1={macro_f1:.3f}\n")
    f.write("\nMicro Average:\n")
    f.write(f"Precision={micro_precision:.3f}, Recall={micro_recall:.3f}, F1={micro_f1:.3f}\n")

print("\nProcessamento finalizado. Relatórios gerados: relatorio_folds.txt, relatorio_final.txt")


Usando dispositivo: cuda
GPU disponível: NVIDIA GeForce RTX 3090

Fold 0
Exemplos few-shot usados: 9
Exemplos few-shot usados para este fold:
Texto: Rapaz é preso por furtos em série Rapaz é preso por furtos em série Texto: Rita de C. Cornélio José Natalino Policarpo , conhecido por " Coquinho ", é acusado de arrombar dois carros. Produtos furtados foram apreendidos Policiais militares prenderam em flagrante, na manhã de ontem, um dos acusados de ter praticado dois furtos durante a madrugada. Vários objetos furtados foram recuperados. O acusado, José Natalino Policarpo , 19 anos , conhecido por " Coquinho ", foi pego após oferecer os produtos furtados para alguns mototáxistas da Bela Vista . Para os policiais da Base Comunitária Noroeste , ele acabou confessando o furto continuado. Policarpo contou aos policiais que havia praticado dois furtos em automóveis, na quadra 35 da rua Saint Martin , e na quadra 35 da rua Araújo Leite . Em contato com as vítimas, elas reconheceram os objetos c

Processando Fold 0:   2%|██▎                                                                                                                                                 | 1/63 [00:28<29:21, 28.41s/it]

Tempo de execução para exemplo: 28.4101 segundos


Processando Fold 0:   3%|████▋                                                                                                                                               | 2/63 [01:09<36:36, 36.02s/it]

Tempo de execução para exemplo: 41.3395 segundos


...

Tempo de execução para exemplo: 14.7434 segundos


Processando Fold 9:  98%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋  | 61/62 [25:52<00:14, 14.91s/it]

Tempo de execução para exemplo: 14.8502 segundos


Processando Fold 9: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 62/62 [26:07<00:00, 25.28s/it]

Tempo de execução para exemplo: 15.0469 segundos

Processamento finalizado. Relatórios gerados: relatorio_folds.txt, relatorio_final.txt


In [4]:
label_list = ner_dataset["train"].features[f"ner_tags"].feature.names
label_list

NameError: name 'ner_dataset' is not defined

In [ ]:
# BERT pretrained model
model_id = 'neuralmind/bert-base-portuguese-cased'

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
import transformers

assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [ ]:
tokenizer("Olá, isto é uma sentença!")

In [ ]:
example = ner_dataset["train"][2]
print(example["tokens"])

In [ ]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True, truncation=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

In [ ]:
len(example[f"ner_tags"]), len(tokenized_input["input_ids"])

In [ ]:
print(tokenized_input.word_ids())

In [ ]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"ner_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

In [ ]:
label_all_tokens = True

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenized_datasets = ner_dataset.map(tokenize_and_align_labels, batched=True, load_from_cache_file=False)

In [ ]:
tokenized_datasets["train"]["labels"][2]

In [ ]:
from transformers import TFAutoModelForTokenClassification

model = TFAutoModelForTokenClassification.from_pretrained(
    model_id, num_labels=len(label_list), from_pt=True
)

In [ ]:
batch_size = 8

In [ ]:
from transformers import create_optimizer

num_train_epochs = 3
num_train_steps = (len(tokenized_datasets["train"]) // batch_size) * num_train_epochs
optimizer, lr_schedule = create_optimizer(
    init_lr=2e-5,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
    num_warmup_steps=0,
)

In [ ]:
import tensorflow as tf

model.compile(optimizer=optimizer)

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer, return_tensors="tf")

In [ ]:
train_set = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)
validation_set = tokenized_datasets["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator,
)
test_set = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator,
)

In [ ]:
import numpy as np
from datasets import load_metric
from transformers.keras_callbacks import KerasMetricCallback

metric = load_metric("seqeval")
labels = [label_list[i] for i in example[f"ner_tags"]]
metric.compute(predictions=[labels], references=[labels])


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


metric_callback = KerasMetricCallback(
    metric_fn=compute_metrics, eval_dataset=validation_set
)

In [ ]:
from transformers.keras_callbacks import PushToHubCallback
# from tensorflow.keras.callbacks import TensorBoard

# model_name = model_checkpoint.split("/")[-1]
# push_to_hub_model_id = f"{model_name}-finetuned-{task}"

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./tc_model_save/logs")

# push_to_hub_callback = PushToHubCallback(
#     output_dir="./tc_model_save",
#     tokenizer=tokenizer,
#     hub_model_id=push_to_hub_model_id,
# )

callbacks = [metric_callback, tensorboard_callback]

model.fit(
    
    train_set,
    validation_data=validation_set,
    epochs=num_train_epochs,
    callbacks=callbacks,
)

In [ ]:
import os

db_path = "optuna_study.db"

if os.path.exists(db_path):
    os.remove(db_path)
    print("Banco de dados deletado com sucesso.")
else:
    print("O banco de dados não existe.")





In [ ]:
import sqlite3

conn = sqlite3.connect("optuna_study.db")
cursor = conn.cursor()

cursor.execute("SELECT * FROM trials")
print(cursor.fetchall())

conn.close()




In [ ]:
model.predict(validation_set)

In [ ]:
model.save_pretrained("./modelNER")

In [ ]:
model.config.id2label.values()

In [ ]:
from datasets import load_metric
import numpy as np


metric = load_metric("seqeval")


def evaluate(model, dataset, ner_labels):
  all_predictions = []
  all_labels = []
  for batch in dataset:
      logits = model.predict(batch)["logits"]
      labels = batch["labels"]
      predictions = np.argmax(logits, axis=-1)
      for prediction, label in zip(predictions, labels):
          for predicted_idx, label_idx in zip(prediction, label):
              if label_idx == -100:
                #   print(label)
                  continue
              all_predictions.append(ner_labels[predicted_idx])
              all_labels.append(ner_labels[label_idx])
              #print('\npredicted=',ner_labels[predicted_idx], '\nlabel=',ner_labels[label_idx])
  #print("all_predictions=",[all_predictions],'\nall_labels=',[all_labels])
  return metric.compute(predictions=[all_predictions], references=[all_labels])

#results = evaluate(model, tf_eval_dataset, ner_labels=list(model.config.id2label.values()))
results = evaluate(model, test_set, ner_labels=list(model.config.id2label.values()))
results